## Creating and training the CNN model 

In [1]:
pwd

'/home/wsuser/work'

In [2]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='EHmhit2MD64AQnqYijN7mrXyaEYoh02jLsiuzU5mzGbt',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

bucket = 'ffdcnnmodelbook-donotdelete-pr-giva0vdmx0opfa'
object_key = 'forestfiredataset.zip'

streaming_body_3 = cos_client.get_object(Bucket=bucket, Key=object_key)['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/


In [3]:
from io import BytesIO
import zipfile
unzip=zipfile.ZipFile(BytesIO(streaming_body_3.read()),'r')
file_paths=unzip.namelist()
for path in file_paths:
    unzip.extract(path)

In [4]:
ls

Dataset/                      fire-classification-model.tgz  forest1.h5
fie-classification-model.tgz  fire.h5


### Import the libraries

In [5]:
import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from matplotlib import pyplot as plt

# Importing ImageDataGenerator from Keras

In [6]:
# image preprocessing (or) image augmentation
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [7]:
#import the cnn layers

# Defining the Parameters

In [8]:
train_datagen = ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True,vertical_flip=True)
#rescale => rescaling pixel value from 0 to 255 to 0 to 1
#shear_range=> counter clock wise rotation(anti clock)

In [9]:
test_datagen = ImageDataGenerator(rescale=1./255)

# Applying ImageDataGenerator functionality to train dataset

In [10]:
x_train = train_datagen.flow_from_directory(r"/home/wsuser/work/Dataset/Dataset/train_set",
                                            target_size=(256,256),
                                            batch_size=32,
                                            class_mode="binary")

Found 436 images belonging to 2 classes.


# Applying ImageDataGenerator functionality to test dataset

In [11]:
x_test = test_datagen.flow_from_directory(r"/home/wsuser/work/Dataset/Dataset/test_set",
                                          target_size=(256,256),
                                          batch_size=32,
                                          class_mode="binary")

Found 121 images belonging to 2 classes.


# Importing Model Building Libraries

In [12]:
from tensorflow.keras.layers import Convolution2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.optimizers import Adam , SGD, RMSprop

In [13]:
x_train.class_indices

{'forest': 0, 'with fire': 1}

# Intializing the model

In [14]:
model = Sequential()

# Adding CNN layers

In [15]:
# add convolution layer

In [16]:
model.add(Convolution2D(32,(3,3),input_shape=(256,256,3),activation="relu"))
# 32 indicates => no of feature detectors
#(3,3)=> kernel size (feature detector size)
#add max pooling layer
model.add(MaxPooling2D(pool_size=(2,2)))
#add flatten layer => input to your ANN  
model.add(Flatten())          

# Add Dense layers

In [17]:
#hidden layer
model.add(Dense(units=300,kernel_initializer="random_uniform",activation="relu"))
model.add(Dense(units=200,kernel_initializer="random_uniform",activation="relu"))
#output layer
model.add(Dense(units=1,kernel_initializer="random_uniform",activation="sigmoid"))

# Configuring the learning process

In [18]:
#compile the model
model.compile(loss=keras.losses.binary_crossentropy,optimizer="adam",metrics=['accuracy'])

# Summarize the model

In [19]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 254, 254, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 127, 127, 32)     0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 516128)            0         
                                                                 
 dense (Dense)               (None, 300)               154838700 
                                                                 
 dense_1 (Dense)             (None, 200)               60200     
                                                                 
 dense_2 (Dense)             (None, 1)                 201       
                                                        

# Training the model

In [20]:
model.fit(x_train,steps_per_epoch=13,epochs=30,validation_data=x_test,validation_steps=3)
#steps_per_epoch = no of training images/batch size
#validation_steps = no of testing images/batch size

Epoch 1/30
13/13 [==============================] - 49s 4s/step - loss: 1.8251 - accuracy: 0.6485 - val_loss: 0.2524 - val_accuracy: 0.8958
Epoch 2/30
13/13 [==============================] - 52s 4s/step - loss: 0.2757 - accuracy: 0.8726 - val_loss: 0.1387 - val_accuracy: 0.9479
Epoch 3/30
13/13 [==============================] - 52s 4s/step - loss: 0.3054 - accuracy: 0.8663 - val_loss: 0.0653 - val_accuracy: 0.9792
Epoch 4/30
13/13 [==============================] - 52s 4s/step - loss: 0.2152 - accuracy: 0.9084 - val_loss: 0.0805 - val_accuracy: 0.9896
Epoch 5/30
13/13 [==============================] - 47s 4s/step - loss: 0.1913 - accuracy: 0.9233 - val_loss: 0.1705 - val_accuracy: 0.9375
Epoch 6/30
13/13 [==============================] - 51s 4s/step - loss: 0.2007 - accuracy: 0.9158 - val_loss: 0.0850 - val_accuracy: 0.9688
Epoch 7/30
13/13 [==============================] - 51s 4s/step - loss: 0.1476 - accuracy: 0.9455 - val_loss: 0.0729 - val_accuracy: 0.9792
Epoch 8/30
13/13 [==

# Saving the model

In [21]:
model.save("fire.h5")

# IBM Deployment

In [2]:
!pip install watson-machine-learning-client

In [3]:
from ibm_watson_machine_learning import APIClient
wml_credentials={
    "url":"https://us-south.ml.cloud.ibm.com",
    "apikey":"1AfypwQwqeHikzD7u4LIKT6DMnD-RPDTyYLRBofzNBPp"
}

client=APIClient(wml_credentials)

In [4]:
client

In [5]:
def guid_space_name(client,fire_deploy):
    space=client.spaces.get_details()
    return(next(item for item in space['resources'] if item['entity']['name']==fire_deploy)['metadata']['id'])

In [6]:
space_uid=guid_space_name(client,'cnn_fire') 
print("Space UID "+space_uid)

Space UID def3a2d0-3dd4-4f16-9ba5-cb9feb7700a1


In [7]:
client.set.default_space(space_uid)

'SUCCESS'

In [28]:
client.software_specifications.list(200)

-------------------------------  ------------------------------------  ----
NAME                             ASSET_ID                              TYPE
default_py3.6                    0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
kernel-spark3.2-scala2.12        020d69ce-7ac1-5e68-ac1a-31189867356a  base
pytorch-onnx_1.3-py3.7-edt       069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6          09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12       09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
pytorch-onnx_rt22.1-py3.9        0b848dd4-e681-5599-be41-b5f6fccc6471  base
ai-function_0.1-py3.6            0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                       0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod     1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6                10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl        111e41b3-de2d-5422-a4d6-bf776828c4b7  base
autoai-kb_rt

In [8]:
software_space_uid=client.software_specifications.get_uid_by_name('tensorflow_rt22.1-py3.9')

In [9]:
software_space_uid

'acd9c798-6974-5d2f-a657-ce06e986df4d'

In [10]:
ls

Dataset/                      fire-classification-model.tgz  forest1.h5
fie-classification-model.tgz  fire.h5


In [11]:
!tar -zcvf fire-classification-model.tgz fire.h5

fire.h5


In [12]:
model_details=client.repository.store_model(model='fire-classification-model.tgz',meta_props={
    client.repository.ModelMetaNames.NAME:"CNN Model Building",
    client.repository.ModelMetaNames.TYPE:'tensorflow_2.7',
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID:software_space_uid
})

In [13]:
model_id=client.repository.get_model_id(model_details)

In [14]:
model_id

'babd0250-5274-4923-850c-7fe9ce7e2409'

In [15]:
client.repository.download(model_id,'fire.tar.gb')

Successfully saved model content to file: 'fire.tar.gb'


'/home/wsuser/work/fire.tar.gb'

In [16]:
ls

Dataset/                      fire-classification-model.tgz  fire.tar.gb
fie-classification-model.tgz  fire.h5                        forest1.h5
